<a href="https://colab.research.google.com/github/sindhujyoti6201/ai-assignment-1/blob/main/assignments/assignment-4/04-assignment-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install requests beautifulsoup4 pydantic openai unified-planning
!pip3 install google-generativeai

# PDDL
The OpenRouter domain is characterized by:

Multiple LLMs
Multiple LLM providers.
Each LLM has capabilities (e.g., coding, multilingual, safe for kids, long context).
Each request has functional requirements (e.g., supports code, must be open-source, etc.)
Each account has a upper limit on the number of dollars it can spend for prompt tokens and completion tokens.
Each LLM has a limit on the number of tokens it can process as part of its context.
Each provider has a cost associated with each LLM expressed in $/m (dollars/million) tokens.
This is not an exhaustive list and therefore you need to embark into an ontology engineering exercise to identify the relevant entities and relationships that will be used to build the PDDL domain and problem files. You can use a simple RAG Agent to help you with this task. You need to produce a structured output where the dictionary keys are the required PDDL specification keywords as shown below. For example the key types will contain all PDDL domain types.

PDDL Domain
Use a free LLM that will search over the OpenRouter documentation pages and extract all relevant types.

Write a PDDL domain file that includes the following:

A set of types that represent the entities in the OpenRouter domain.
A set of predicates that represent the relationships between the entities.
A set of actions that represent the operations that can be performed in the OpenRouter domain. Each action should have preconditions and effects that are relevant to the OpenRouter domain.
PDDL Problem
Write a PDDL problem file that includes the following: * A set of objects that represent the specific instances of the types defined in the domain file. * A set of init predicates that represent the initial state of the OpenRouter domain. This should include the available LLMs, providers, and their capabilities. * A set of goal predicates that represent the desired state of the OpenRouter domain. This should include the successful routing of requests to the appropriate LLMs based on their capabilities and costs.

Solve the PDDL problem using an appropriate PDDL solver provided by the Unified Planning library and use its ability to read and write PDDL files.

Note
You are free to use relevant VSCode plugins to help you with the PDDL domain and problem statement as well as solver calls, however, the agent must be in its entirety implemented in Python.

# EXTRACTION OF DATA FROM THE OPENROUTER APIs

In [ ]:
import requests
import json

def fetch_and_save_models_data():
    url = "https://openrouter.ai/api/v1/models"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()['data']
        with open("models_data.json", "w") as f:
            json.dump(data, f, indent=2)
        print("models_data.json saved.")
    else:
        raise Exception(f"Failed to fetch models. Status code: {response.status_code}")

fetch_and_save_models_data()

models_data.json saved.


# MINIMIZING AND EXTRACTING KEY/REQUIRED FEATURES

In [ ]:
from openai import OpenAI
import os, re

def summarize_model_data(input_file="models_data.json", output_file="models_summary.json"):
    with open(input_file, "r") as f:
        models = json.load(f)

    prompt = f"""
You are an expert data extraction assistant trained to analyze complex model metadata from OpenRouter.
Your task is to extract only the most relevant attributes for routing large language model (LLM) requests.

The input is a JSON array of detailed model descriptions. From each model object, extract and return the following fields in a clean JSON array:

- model id (string): The unique model identifier (e.g., "meta-llama/llama-guard-4-12b")
- model name (string): Human-readable name
- context_length (integer): The maximum context tokens allowed
- input_modalities (list of strings): Accepted input types (e.g., text, image)
- output_modalities (list of strings): Output types generated by the model
- capabilities (list of strings or inferred tags): Such as coding, multilingual, safety, reasoning, etc.
- pricing (object): Including prompt and completion cost in $/token (extract only those two fields)
- supported_parameters (list of strings): The list of tunable parameters this model supports

Be sure to only extract what's required, and maintain a compact, flat structure in the output JSON.

Here is the input JSON data:
{json.dumps(models[:10])}

Please just output the JSON array. Do not include any additional text or explanation.
"""

    client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key='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')

    response = client.chat.completions.create(
        model="meta-llama/llama-3.3-70b-instruct:free",
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": "You are a data summarizer for model routing."}]
    )

    if response and response.choices and response.choices[0].message:
        raw_output = response.choices[0].message.content.strip()

        # Extract all text between the first triple backticks block (if exists)
        try:
            match = re.search(r"```(?:json)?\\s*(.*?)```", raw_output, re.DOTALL)
            json_str = match.group(1).strip() if match else raw_output

            parsed_json = json.loads(json_str)
            with open(output_file, "w") as f:
                json.dump(parsed_json, f, indent=2)
            print("models_summary.json saved.")
        except Exception as e:
            print("Failed to parse JSON from model response.")
            print("Raw output:", raw_output)
            print("Error:", str(e))
    else:
        print("No valid response received from the model.")
        print("Raw response:", response)

summarize_model_data()

models_summary.json saved.


# GENERATION OF PDDL FILES

# DOMAIN.PDDL USING OPENROUTER MAVERICK MODEL

In [ ]:
def generate_domain_pddl(summary_path="models_summary.json"):
    with open(summary_path, "r") as f:
        summary = f.read()

    prompt = f"""
You are an expert in PDDL (Planning Domain Definition Language) and Pyperplan. You are given a task to create a PDDL domain file for routing AI model requests through OpenRouter. The domain should be compatible with Pyperplan.
Using the following model metadata:
{summary}

Generate a complete and syntactically correct PDDL domain file in Pyperplan compatible format for routing AI model requests through OpenRouter. Your output should include:

1. (:types): Define the types: llm, provider, capability, request, account. etc...
2. (:predicates): Define logical predicates such as:
   - (has-capability ?m - llm ?c - capability)
   - (belongs-to ?m - llm ?p - provider)
   - (has-cost ?m - llm ?d - cost)
   - (has-token-limit ?m - llm ?t - tokenlimit)
   - (meets-requirement ?r - request ?c - capability)
   - (can-handle ?m - llm ?r - request)
   and other logical predicates as needed.
3. (:action route-request):
   - Parameters: (?r - request ?m - llm)
   - Preconditions: The llm has required capability, cost, and context token limits to satisfy request
   - Effects: The request is routed to the llm
   and other necessary actions and effects.

4. FOR EVERY variable(?l) define its type like ?l - llm -- FOLLOW THESE IMPORTANT
5. There should be following prdeicates in the domain file as a mandatory:
    (fits-request ?l - llm ?r - request)
    (meets-requirement ?r - request ?c - capability)
    (not-processed ?r - request)
    (processed ?r - request)
    (routed ?r - request ?l - llm)
6. There should be the following actions in the domain file as a mandatory:
    (:action route-request
    :parameters (?r - request ?l - llm)
    :precondition (and
      (not-processed ?r)
      (fits-request ?l ?r)
    )
    :effect (and
      (not (not-processed ?r))
      (processed ?r)
      (routed ?r ?l)
    )
  )
7. ONLY USE THOSE OBJECTS, ACTIONS AND PREDICATES THAT ARE DEFINED IN THE objects.

RULES:
Rules for generating STRIPS-compatible PDDL:

1. All PDDL files must follow valid structure: domain files require :types, :predicates, and :action definitions; problem files require :domain, :objects, :init, and :goal blocks.
2. Only define types in :types, and use them in :parameters and :objects — do NOT specify types inside predicate calls.
3. Every predicate used in :init, :precondition, :effect, or :goal must be declared in :predicates.
4. Inside :parameters, define variable types like (?r - request ?l - llm), but inside expressions like (has-capability ?l ?c), do NOT repeat types.
5. STRIPS does not support arithmetic or relational operators like <=, >=, <, >, =. Do not use expressions like (<= ?x ?y) or (+ ?a ?b). Instead, precompute and encode relationships symbolically using named predicates (e.g., (fits-context-length llm1 req1)).
6. Do not use :functions or numeric fluents. STRIPS only allows Boolean predicates.
7. Every variable in :parameters must be used somewhere in the :precondition or :effect of the action.
8. Every object or constant referenced in :init or :goal must be listed in the :objects section of the problem file.
9. Avoid existential or universal quantifiers (exists, forall) — these are not supported in basic STRIPS planners.
10. To check conditions like token limits, cost, or context length, encode them symbolically as precomputed predicates in :init (e.g., (fits-request llm req)).
11. Effects can only add or delete atomic predicates — no nested expressions or arithmetic.
12. Use clear naming conventions for objects, types, and predicates to avoid name collisions and parsing issues.


Other naming RULES:
Naming Rules for Valid PDDL:
1. Do not use hyphens (`-`) in the names of objects, constants, types, or predicates. Hyphens are reserved for type declarations in PDDL and will break parsing if used elsewhere.
2. Use underscores (`_`) instead of hyphens for compound names. For example, use `tool_use` instead of `tool-use`.
3. Object names must be unique within their scope and should not conflict across different types (e.g., avoid using the same name for both an inputmodality and outputmodality).
4. All names must begin with a lowercase letter and may contain only lowercase letters, digits, and underscores.
5. Avoid names that resemble numbers (e.g., `n0-0000025`) unless clearly distinguishable from numeric literals. Use formats like `n0000025` or `n_25` instead.
6. Do not enclose names in quotes.
7. Ensure that every name used in `:init`, `:goal`, `:precondition`, or `:effect` is declared under `:objects` in the problem file or as a parameter in an action.


Sample domain file:
(define (domain openrouter-routing)
  (:requirements :typing)

  (:types
    llm provider capability request account number
  )

  (:predicates
    (has-capability ?l - llm ?c - capability)
    (provided-by ?l - llm ?p - provider)
    (cost-prompt ?l - llm ?n - number)
    (cost-completion ?l - llm ?n - number)
    (context-length ?l - llm ?n - number)
    (request-length ?r - request ?n - number)
    (within-budget ?a - account ?n - number)
    (fits-request ?l - llm ?r - request)
    (meets-requirement ?r - request ?c - capability)
    (not-processed ?r - request)
    (processed ?r - request)
    (routed ?r - request ?l - llm)
  )

  (:action route-request
    :parameters (?r - request ?l - llm)
    :precondition (and
      (not-processed ?r)
      (fits-request ?l ?r)
    )
    :effect (and
      (not (not-processed ?r))
      (processed ?r)
      (routed ?r ?l)
    )
  )
)


ANOTHER SAMPLE DOMAIN FILE:
(define (domain openrouter-routing)
  (:requirements :strips :typing)

  (:types
    llm provider capability request account tokenlimit cost number inputmodality outputmodality parameter
  )

  (:predicates
    (has-capability ?l - llm ?c - capability)
    (provided-by ?l - llm ?p - provider)
    (has-cost-prompt ?l - llm ?n - number)
    (has-cost-completion ?l - llm ?n - number)
    (has-token-limit ?l - llm ?t - tokenlimit)
    (request-token-length ?r - request ?n - number)
    (request-input-modality ?r - request ?i - inputmodality)
    (request-output-modality ?r - request ?o - outputmodality)
    (request-parameter ?r - request ?p - parameter)
    (llm-input-modality ?l - llm ?i - inputmodality)
    (llm-output-modality ?l - llm ?o - outputmodality)
    (llm-supported-parameter ?l - llm ?p - parameter)
    (within-budget ?a - account ?n - number)
    (meets-requirement ?r - request ?c - capability)
    (fits-request ?l - llm ?r - request)
    (not-processed ?r - request)
    (processed ?r - request)
    (routed ?r - request ?l - llm)
  )

  (:action route-request
    :parameters (?r - request ?l - llm)
    :precondition (and
      (not-processed ?r)
      (fits-request ?l ?r)
    )
    :effect (and
      (not (not-processed ?r))
      (processed ?r)
      (routed ?r ?l)
    )
  )
)

TRY TO KEEP THE DOMAIN FILE AS SIMILAR AS POSSIBLE TO THE EXAMPLE.

This is a sample domain file. You can use it as a reference to create your own domain file based on the provided model metadata.OUTPUT ONLY THE DOMAIN FILE IN PDDL FORMAT. NO ADDITIONAL TEXT OR EXPLANATION. NO CODE BLOCKS LIKE ```lisp```.
"""

    client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key='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')
    response = client.chat.completions.create(
        model="meta-llama/llama-4-maverick:free",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "You are a PDDL generation assistant. Your task is to generate a valid domain.pddl file for use with the ENHSP planner. Use structured types and logical predicates to represent an AI model routing system."}
        ]
    )

    pddl_code = response.choices[0].message.content.strip()
    with open("domain.pddl", "w") as f:
        f.write(pddl_code)
    print("domain.pddl saved.")


generate_domain_pddl()

domain.pddl saved.


# PROBLEM.PDDL USING OPENROUTER MAVERICK MODEL

In [ ]:
def generate_problem_pddl(summary_path="models_summary.json", query_request = "I want to route one request req1 to a multilingual and coding-capable request for a free user, other request req2 to safe-for-kids", comment=""):
    with open(summary_path, "r") as f:
        summary = f.read()

    with open("domain.pddl", "r") as f:
        domain = f.read()

    print(domain)


    prompt = f"""
You are an expert in PDDL (Planning Domain Definition Language) and Pyperplan. You are given a task to create a PDDL problem file for routing AI model requests through OpenRouter. The problem should be compatible with Pyperplan.
Using the following model metadata:
{summary}
for the domain {domain}. Try to fix the error mention in the comment = {comment}

This problem should contain the all the necessary objects, initial conditions, and goals to route requests to the appropriate LLMs based on their capabilities and requirements for the query request = {query_request}.

GENERATE A PROBLEM.PDDL FILE IN PYPPERPLAN COMPATIBLE FORMAT USING {domain} and {query_request} such that there exists a plan.

Before proceeding to creating the problem file, please do the following:
1. Extract all the requests from the {query_request} and create unique request objects for each. The {query_request} should contain a requests with their requirements in a comma separated format.
2. Create unique capability objects for each capability mentioned in the {query_request}.
3. Create unique LLM objects for each LLM mentioned in the {query_request} and assign them to the appropriate provider.
The problem should be structured as follows:

1. Each requiring a unique combination of capabilities like 'coding', 'multilingual', or 'safe-for-kids'.
2. Each request should:
   - Be declared as an object of type `request`
   - Include its requirements using (meets-requirement ?r - request ?c - capability)
   - Have an initial status of (not-processed ?r)
3. THE INIT SECTION IS ALWAYS FILLED IN ACCORDANCE WITH THE FOLLWOING RULES:
Rules for Filling the :init Section in PDDL:

1. Only include ground (fully instantiated) predicates — i.e., use concrete object names, no variables.
2. Ensure that each predicate in :init exactly matches a predicate declared in the domain = {domain}, in both name and argument count.
3. The types of each object used must match the argument types defined for that predicate in the domain.
4. All objects referenced in :init must first be declared in the :objects section with their correct types, as specified in the domain.
5. Do not use hyphens in object names — use underscores instead (e.g., use `safe_for_kids` not `safe-for-kids`).
6. Only use Boolean predicates (e.g., `(has-capability llm1 multilingual)`), and avoid any numeric expressions, functions, or comparison operators like `+`, `=`, `<=`, etc.
7. If a predicate expects an object of type `number`, then the object (e.g., `n0`) must be declared as `- number` in the :objects section — same for all custom types like `modality`, `parameter`, etc.
8. Do not duplicate facts — ensure each fact is unique in :init.
9. If derived predicates like `(fits-request ?l ?r)` or `(meets-requirement ?r ?c)` are required and not computed via actions, include them explicitly in :init.
10. Follow the exact type names, object names, and predicate formats defined in the domain file. Inconsistent usage will cause parsing errors.

Example of valid :init usage:
(:init
  (has-capability qwen_qwen3_0_6b_free multilingual)
  (provided-by qwen_qwen3_0_6b_free openrouter)
  (has-cost-prompt qwen_qwen3_0_6b_free n0)
  (has-token-limit qwen_qwen3_0_6b_free tokenlimit_32000)
  (llm-input-modality qwen_qwen3_0_6b_free text_input)
  (llm-output-modality qwen_qwen3_0_6b_free text_output)
  (request-input-modality req1 text_input)
  (request-output-modality req1 text_output)
  (not-processed req1)
)



3. Include a set of LLMs (up to 5) and their capabilities, costs, and context limits in the init section.
4. Include a goal section that requires all requests to be routed (i.e., they are no longer not-processed).
5. Use object declarations for types llm, capability, request, provider, and account.
6. Don't use any floating points, numbers, etc .Evrything should be text.Donot use . or the decimal in pddl. Wherever you see a decimal, replace it with a dash (-). For exaample 0.5 should be replaced with 0-5, qwen3.0.6b becomes qwen3-0-6b.
7. Goals should be route requests or processed requests. like processed req1. Don't Decide by yourself which llm to route to.
8. ONLY USE THOSE OBJECTS, ACTIONS AND PREDICATES THAT ARE DEFINED IN THE {domain}.
9. VERY IMPORTANT - The Goals should always be in this FORMAT.
(:goal
  (and
    (processed req1)
    (processed req2)
    (processed req3)
    (processed req4)..
  )
)


Few CAUTIONS VERY IMPORTANT FOR THE PDDL FILE USING PYPPERPLAN:
- Do not use the following constructs in the PDDL file:
1. Numeric fluents and expressions like (<, >, +, -, *, /)
2. Conditional effects like (exists, forall, imply, leq)
3. Donot use . or the decimal in pddl. Wherever you see a decimal, replace it with a dash (-). For exaample 0.5 should be replaced with 0-5, qwen3.0.6b becomes qwen3-0-6b
4. JUST output the domain file in PDDL format. No additional text or explanations. Nor should you use any code blocks like ```lisp or ```.
5. DONOT USE BUILT IN FUNCTIONS LIKE forall, imply, or, max-number in the PDDL file. These don't support Pyperplan.
6. DON't DEFINE SAME NAME IN THE OBJECTS- like the following:
text image - inputmodality
    text - outputmodality.

    This should be :
text-input image - inputmodality
text-output - outputmodality
7. Other naming RULES:
Naming Rules for Valid PDDL:
1. Do not use hyphens (`-`) in the names of objects, constants, types, or predicates. Hyphens are reserved for type declarations in PDDL and will break parsing if used elsewhere.
2. Use underscores (`_`) instead of hyphens for compound names. For example, use `tool_use` instead of `tool-use`.
3. Object names must be unique within their scope and should not conflict across different types (e.g., avoid using the same name for both an inputmodality and outputmodality).
4. All names must begin with a lowercase letter and may contain only lowercase letters, digits, and underscores.
5. Avoid names that resemble numbers (e.g., `n0-0000025`) unless clearly distinguishable from numeric literals. Use formats like `n0000025` or `n_25` instead.
6. Do not enclose names in quotes.
7. Ensure that every name used in `:init`, `:goal`, `:precondition`, or `:effect` is declared under `:objects` in the problem file or as a parameter in an action.


Sample problem file:
(define (problem route-llm-request)
  (:domain openrouter-routing)

  (:objects
    qwen3-0-6b qwen3-1-7b - llm
    p1 - provider
    cap1 cap2 - capability
    req1 req2 - request
    acc1 - account
    n0 n1 n2 n3 n4 - number
  )

  (:init
    (has-capability qwen3-0-6b cap1)
    (has-capability qwen3-1-7b cap2)
    (provided-by qwen3-0-6b p1)
    (cost-prompt qwen3-0-6b n1)
    (cost-completion qwen3-0-6b n2)
    (context-length qwen3-0-6b n3)
    (request-length req1 n1)
    (within-budget acc1 n4)
    (meets-requirement req1 cap1)
    (not-processed req1)
    (fits-request qwen3-0-6b req1)
  )

  (:goal
    (and
      (processed req1)
      (processed req2)
      (processed req3)
      (processed req4)
    )
  )
)

There should always be some fits-requests in the init section.
Example:
(fits-request qwen_qwen3_0_6b_free req1)
(fits-request deepseek_deepseek_prover_v2_free req2)

Eaxmple 2:
(define (problem route-llm-request)
  (:domain openrouter-routing)

  (:objects
    qwen_qwen3_30b_a3b_free meta_llama_llama_guard_4_12b - llm
    openrouter - provider
    multilingual coding content_safety - capability
    req1 req2 - request
    acc1 - account
    n0 n1 - number
    tokenlimit_40960 tokenlimit_163840 - tokenlimit
    text_input image_input - inputmodality
    text_output - outputmodality
    max_tokens temperature - parameter
  )

  (:init
    (has-capability qwen_qwen3_30b_a3b_free multilingual)
    (has-capability qwen_qwen3_30b_a3b_free coding)
    (has-capability meta_llama_llama_guard_4_12b content_safety)
    (provided-by qwen_qwen3_30b_a3b_free openrouter)
    (provided-by meta_llama_llama_guard_4_12b openrouter)
    (has-cost-prompt qwen_qwen3_30b_a3b_free n0)
    (has-cost-completion qwen_qwen3_30b_a3b_free n0)
    (has-cost-prompt meta_llama_llama_guard_4_12b n1)
    (has-cost-completion meta_llama_llama_guard_4_12b n1)
    (has-token-limit qwen_qwen3_30b_a3b_free tokenlimit_40960)
    (has-token-limit meta_llama_llama_guard_4_12b tokenlimit_163840)
    (llm-input-modality qwen_qwen3_30b_a3b_free text_input)
    (llm-output-modality qwen_qwen3_30b_a3b_free text_output)
    (llm-input-modality meta_llama_llama_guard_4_12b image_input)
    (llm-input-modality meta_llama_llama_guard_4_12b text_input)
    (llm-output-modality meta_llama_llama_guard_4_12b text_output)
    (llm-supported-parameter qwen_qwen3_30b_a3b_free max_tokens)
    (llm-supported-parameter qwen_qwen3_30b_a3b_free temperature)
    (llm-supported-parameter meta_llama_llama_guard_4_12b max_tokens)
    (llm-supported-parameter meta_llama_llama_guard_4_12b temperature)
    (request-input-modality req1 text_input)
    (request-output-modality req1 text_output)
    (request-input-modality req2 text_input)
    (request-output-modality req2 text_output)
    (request-input-modality req2 image_input)
    (meets-requirement req1 multilingual)
    (meets-requirement req1 coding)
    (meets-requirement req2 content_safety)
    (not-processed req1)
    (not-processed req2)
    (within-budget acc1 n0)
    (fits-request qwen_qwen3_30b_a3b_free req1)
    (fits-request meta_llama_llama_guard_4_12b req2)
  )

  (:goal
    (and
      (processed req1)
      (processed req2)
    )
  )
)

Example 3:
(define (problem route-llm-request)
  (:domain openrouter-routing)

  (:objects
    qwen_qwen3_0_6b_free qwen_qwen3_1_7b_free qwen_qwen3_4b_free qwen_qwen3_30b_a3b_free deepseek_deepseek_prover_v2_free meta_llama_llama_guard_4_12b - llm
    openrouter - provider
    multilingual coding content_safety reasoning - capability
    req1 req2 - request
    acc1 - account
    n0 n1 - number
    tokenlimit_32000 tokenlimit_128000 tokenlimit_40960 tokenlimit_163840 - tokenlimit
    text_input image_input - inputmodality
    text_output - outputmodality
    max_tokens temperature - parameter
  )

  (:init
    (has-capability qwen_qwen3_0_6b_free multilingual)
    (has-capability qwen_qwen3_0_6b_free reasoning)
    (has-capability qwen_qwen3_1_7b_free multilingual)
    (has-capability qwen_qwen3_1_7b_free reasoning)
    (has-capability qwen_qwen3_4b_free reasoning)
    (has-capability qwen_qwen3_30b_a3b_free multilingual)
    (has-capability qwen_qwen3_30b_a3b_free coding)
    (has-capability deepseek_deepseek_prover_v2_free reasoning)
    (has-capability meta_llama_llama_guard_4_12b content_safety)

    (provided-by qwen_qwen3_0_6b_free openrouter)
    (provided-by qwen_qwen3_1_7b_free openrouter)
    (provided-by qwen_qwen3_4b_free openrouter)
    (provided-by qwen_qwen3_30b_a3b_free openrouter)
    (provided-by deepseek_deepseek_prover_v2_free openrouter)
    (provided-by meta_llama_llama_guard_4_12b openrouter)

    (has-cost-prompt qwen_qwen3_0_6b_free n0)
    (has-cost-completion qwen_qwen3_0_6b_free n0)
    (has-cost-prompt qwen_qwen3_1_7b_free n0)
    (has-cost-completion qwen_qwen3_1_7b_free n0)
    (has-cost-prompt qwen_qwen3_4b_free n0)
    (has-cost-completion qwen_qwen3_4b_free n0)
    (has-cost-prompt qwen_qwen3_30b_a3b_free n0)
    (has-cost-completion qwen_qwen3_30b_a3b_free n0)
    (has-cost-prompt deepseek_deepseek_prover_v2_free n0)
    (has-cost-completion deepseek_deepseek_prover_v2_free n0)
    (has-cost-prompt meta_llama_llama_guard_4_12b n1)
    (has-cost-completion meta_llama_llama_guard_4_12b n1)

    (has-token-limit qwen_qwen3_0_6b_free tokenlimit_32000)
    (has-token-limit qwen_qwen3_1_7b_free tokenlimit_32000)
    (has-token-limit qwen_qwen3_4b_free tokenlimit_128000)
    (has-token-limit qwen_qwen3_30b_a3b_free tokenlimit_40960)
    (has-token-limit deepseek_deepseek_prover_v2_free tokenlimit_163840)
    (has-token-limit meta_llama_llama_guard_4_12b tokenlimit_163840)

    (llm-input-modality qwen_qwen3_0_6b_free text_input)
    (llm-output-modality qwen_qwen3_0_6b_free text_output)
    (llm-input-modality qwen_qwen3_1_7b_free text_input)
    (llm-output-modality qwen_qwen3_1_7b_free text_output)
    (llm-input-modality qwen_qwen3_4b_free text_input)
    (llm-output-modality qwen_qwen3_4b_free text_output)
    (llm-input-modality qwen_qwen3_30b_a3b_free text_input)
    (llm-output-modality qwen_qwen3_30b_a3b_free text_output)
    (llm-input-modality deepseek_deepseek_prover_v2_free text_input)
    (llm-output-modality deepseek_deepseek_prover_v2_free text_output)
    (llm-input-modality meta_llama_llama_guard_4_12b text_input)
    (llm-input-modality meta_llama_llama_guard_4_12b image_input)
    (llm-output-modality meta_llama_llama_guard_4_12b text_output)

    (llm-supported-parameter qwen_qwen3_0_6b_free max_tokens)
    (llm-supported-parameter qwen_qwen3_0_6b_free temperature)
    (llm-supported-parameter qwen_qwen3_1_7b_free max_tokens)
    (llm-supported-parameter qwen_qwen3_1_7b_free temperature)
    (llm-supported-parameter qwen_qwen3_4b_free max_tokens)
    (llm-supported-parameter qwen_qwen3_4b_free temperature)
    (llm-supported-parameter qwen_qwen3_30b_a3b_free max_tokens)
    (llm-supported-parameter qwen_qwen3_30b_a3b_free temperature)
    (llm-supported-parameter deepseek_deepseek_prover_v2_free max_tokens)
    (llm-supported-parameter deepseek_deepseek_prover_v2_free temperature)
    (llm-supported-parameter meta_llama_llama_guard_4_12b max_tokens)
    (llm-supported-parameter meta_llama_llama_guard_4_12b temperature)

    (request-input-modality req1 text_input)
    (request-output-modality req1 text_output)
    (request-input-modality req2 text_input)
    (request-input-modality req2 image_input)
    (request-output-modality req2 text_output)

    (meets-requirement req1 multilingual)
    (meets-requirement req1 coding)
    (meets-requirement req2 content_safety)

    (not-processed req1)
    (not-processed req2)

    (within-budget acc1 n0)
    (fits-request qwen_qwen3_30b_a3b_free req1)
    (fits-request meta_llama_llama_guard_4_12b req2)
  )

  (:goal
    (and
      (processed req1)
      (processed req2)
    )
  )
)

Try to fill for all the predicates mentioned in the {domain} in the init section.

This is a sample problem file. You can use it as a reference to create your own domain file based on the provided model metadata.
Make sure to include all the necessary predicates and actions that are relevant to the routing of AI model requests through OpenRouter. The domain file should be complete and ready for use with Pyperplan.
Please don't include any additional text or explanation in the problem file. Nor should you use any code blocks like ```lisp or ```.

Use consistent, lowercase, underscore-separated names for all PDDL objects, starting with a letter and avoiding malformed fragments (e.g., use qwen_qwen3_4b_free instead of q_3_4b_free), strictly matching the types and format declared in the domain.
"""

    client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key='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')
    response = client.chat.completions.create(
        model="meta-llama/llama-4-maverick:free",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": "You are a PDDL generation assistant. Your task is to generate a valid problem.pddl file compatible with Pyperplan based on provided domain constraints and object definitions. remove ```lisp from the start of the file and ``` from the end of the file."}
        ]
    )

    pddl_code = response.choices[0].message.content.strip()
    with open("problem.pddl", "w") as f:
        f.write(pddl_code)
    print("problem.pddl saved.")

generate_problem_pddl()

(define (domain openrouter-routing)
  (:requirements :strips :typing)

  (:types
    llm provider capability request account tokenlimit cost number inputmodality outputmodality parameter
  )

  (:predicates
    (has_capability ?l - llm ?c - capability)
    (belongs_to ?l - llm ?p - provider)
    (has_cost_prompt ?l - llm ?n - number)
    (has_cost_completion ?l - llm ?n - number)
    (has_token_limit ?l - llm ?t - tokenlimit)
    (request_token_length ?r - request ?n - number)
    (request_input_modality ?r - request ?i - inputmodality)
    (request_output_modality ?r - request ?o - outputmodality)
    (request_parameter ?r - request ?p - parameter)
    (llm_input_modality ?l - llm ?i - inputmodality)
    (llm_output_modality ?l - llm ?o - outputmodality)
    (llm_supported_parameter ?l - llm ?p - parameter)
    (within_budget ?a - account ?n - number)
    (meets_requirement ?r - request ?c - capability)
    (fits_request ?l - llm ?r - request)
    (not_processed ?r - request)
    (pro

# SOLVERS

In [ ]:
!pip install unified-planning[planners]
!pip install unified-planning[enhsp]
!pip install unified-planning[pyperplan]


In [ ]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
def solve_pddl():
    reader = PDDLReader()
    problem = reader.parse_problem('/content/domain.pddl', '/content/problem.pddl')

    try:
      print(f"Using to Pyperplan solver")
      with OneshotPlanner(name='pyperplan') as planner:
          result = planner.solve(problem)
          print(f"Planner used: {planner.name}")
          print(f"Result Status: {result.status}")

      if result.plan:
          res = "Solved PDDL. The plan is:\n"
          print("Plan Found:")
          for action in result.plan.actions:
              res += str(action) + "\n"
          return res
      else:
          print("No plan found.")
          return "No plan found."
    except Exception as e:
      return str(e)



print("==============================DOMAIN.PDDL=============================")
with open("domain.pddl", "r") as f:
    print(f.read())

print("==============================PROBLEM.PDDL=============================")
with open("problem.pddl", "r") as f:
    print(f.read())

print("==============================PLAN=============================")
solve_pddl()

==============================DOMAIN.PDDL=============================
(define (domain openrouter-routing)
  (:requirements :strips :typing)

  (:types
    llm provider capability request account tokenlimit cost number inputmodality outputmodality parameter
  )

  (:predicates
    (has-capability ?l - llm ?c - capability)
    (provided-by ?l - llm ?p - provider)
    (has-cost-prompt ?l - llm ?n - number)
    (has-cost-completion ?l - llm ?n - number)
    (has-token-limit ?l - llm ?t - tokenlimit)
    (request-token-length ?r - request ?n - number)
    (request-input-modality ?r - request ?i - inputmodality)
    (request-output-modality ?r - request ?o - outputmodality)
    (request-parameter ?r - request ?p - parameter)
    (llm-input-modality ?l - llm ?i - inputmodality)
    (llm-output-modality ?l - llm ?o - outputmodality)
    (llm-supported-parameter ?l - llm ?p - parameter)
    (within-budget ?a - account ?n - number)
    (meets-requirement ?r - request ?c - capability)
    (fits-

'Solved PDDL. The plan is:\nroute-request(req2, deepseek_deepseek_prover_v2_free)\nroute-request(req1, qwen_qwen3_1_7b_free)\n'

# ReAct Agent Implementation
The previous step showcased a feedforward apprach where LLMs where used to help you craft the PDDL domain and problem files. Here we are going to incorporate it into a ReAct agent that will be able to reason and act in a step-by-step manner, often reacting depending on the feedback we receive from the environment.

You must use the Model Context Protocol (MCP) servers to structure the tool calling component of the agent. Tools are external to the agent APIs that can help it achieve its goal. For example, the calls to the Unified Planning library may be considered a tool that the agent can call when it needs to.

Your agent should be able to interact with an environment that simulates the OpenRouter API and therefore provides the necessary information for the agent to make decisions such as deviating from its default routing policy. Alternatively, you may use the OpenRouter API directly if you prefer but limit yourselves to free models and include additional information that will necessitate routing policy change such as dynamic pricing.


In [ ]:
def run_pipeline_for_pddl_files_generation(query, comment):
    try:
        fetch_and_save_models_data()
        summarize_model_data()
        generate_domain_pddl()
        generate_problem_pddl(query, comment)

        return "Pipeline completed successfully."
    except Exception as e:
        return str(e)


In [ ]:
!pip install groq

In [ ]:
from groq import Groq
import os
import re

# Setup client
client = Groq(api_key="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")


# === ReAct Agent ===
class ReActAgent:
    def __init__(self, client, system=""):
        self.client = client
        self.system = system
        self.messages = []
        self.initial_query = ""
        self.last_generate_argument = ""

        if self.system:
            self.messages.append({"role": "system", "content": self.system})

        self.tools = {
            "generate_pddl": run_pipeline_for_pddl_files_generation,
            "solve_pddl": solve_pddl
        }

    def run(self, query: str, max_steps: int = 10):
        print(f"Running ReAct agent for query: '{query}'")
        self.initial_query = query
        next_prompt = query

        for i in range(max_steps):
            self.messages.append({"role": "user", "content": next_prompt})
            result = self._call_model()
            print("-------- MODEL RESPONSE --------")
            print(result)
            print("--------------------------------")

            if "Answer:" in result:
                return result

            if "Action:" in result and "PAUSE" in result:
                action_match = re.search(r"Action:\s*(\w+):", result)
                if not action_match:
                    next_prompt = "Observation: Invalid action"
                    print(f"<<< {next_prompt}")
                    continue

                action = action_match.group(1).strip()
                argument = ""

                if action == "generate_pddl":
                    arg_match = re.search(r"Action:\s*generate_pddl:\s*(.*)", result)
                    if arg_match:
                        argument = arg_match.group(1).strip()
                        self.last_generate_argument = argument
                elif action == "solve_pddl":
                    argument = ""

                print(f"Action: {action}, Argument: {argument}")

                try:
                    if action == "generate_pddl":
                        result_tool = self.tools[action](argument)
                    elif action == "solve_pddl":
                        try:
                            result_tool = self.tools[action]()
                        except Exception as e:
                            # Retry generate_pddl with error context
                            retry_comment = f"[RETRY due to error: {str(e)}]"
                            print("Solve failed. Regenerating PDDL with error context...")
                            self.tools["generate_pddl"](self.last_generate_argument, retry_comment)
                            print("Retrying solve...")
                            result_tool = self.tools["solve_pddl"]()
                    else:
                        result_tool = "Tool not found"

                except Exception as e:
                    result_tool = f"Tool {action} failed with error: {e}"

                next_prompt = f"Observation: {result_tool}"
                print(f">>> {next_prompt}")
                continue

            break

    def _call_model(self):
        completion = self.client.chat.completions.create(
            model="llama3-70b-8192",
            messages=self.messages,
            temperature=0
        )
        content = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": content})
        return content


# === ReAct Prompt ===

react_system_prompt = """
You are a ReAct agent operating in a loop of: Thought → Action → PAUSE → Observation.
At the end, output a final Answer.

Use:
- Thought: Describe what you're thinking.
- Action: Call an available tool with: tool_name: input
- PAUSE: Always follows an action.
- Observation: Will be returned as a user message.

Available tools:
- generate_pddl: <task_description>
- solve_pddl: <leave blank>

The following is only for example purposes. DONOT USE IT IN PRODUCTION.
Example or sample:

Question: I want to solve a planning problem for OpenRouter LLM routing.

Thought: I need to generate the PDDL files that define the domain and problem.
Action: generate_pddl: OpenRouter LLM routing problem
PAUSE

Observation: PDDL files generated for: OpenRouter LLM routing problem

Thought: Now I should solve this PDDL problem using a planning tool.
Action: solve_pddl:
PAUSE

Observation: Solved PDDL problem. Plan: (select-llm qwen3-14b user-request)

Answer: The request was routed successfully using the plan: (select-llm qwen3-14b user-request)
""".strip()


# === Run the Agent ===

agent = ReActAgent(client, system=react_system_prompt)
agent.run("I want to route a multilingual and coding-capable request for a free user.")

Running ReAct agent for query: 'I want to route a multilingual and coding-capable request for a free user.'
-------- MODEL RESPONSE --------
Thought: I need to generate the PDDL files that define the domain and problem for this request.

Action: generate_pddl: multilingual and coding-capable request for a free user

PAUSE
--------------------------------
Action: generate_pddl, Argument: multilingual and coding-capable request for a free user
>>> Observation: Tool generate_pddl failed with error: run_pipeline_for_pddl_files_generation() missing 1 required positional argument: 'comment'
-------- MODEL RESPONSE --------
Thought: It seems like the generate_pddl tool requires more information, specifically a comment about the request.

Action: generate_pddl: multilingual and coding-capable request for a free user, comment: route a free user request that supports multiple languages and coding capabilities

PAUSE
--------------------------------
Action: generate_pddl, Argument: multilingual a

'Thought: It looks like the solve_pddl tool was able to successfully route the request.\n\nAnswer: The request was routed successfully using the plan: route-request(req2, deepseek_deepseek_prover_v2_free) route-request(req1, qwen_qwen3_1_7b_free)'

# ReAct Agent Explanation:

The provided Python code implements a ReAct agent, an AI model that solves problems by thinking step-by-step and performing actions. It processes user queries by first generating a response (a "Thought"), then deciding if it needs to execute a tool (an "Action"), pausing to wait for the result, and finally using the observation from that tool to continue its reasoning. The agent repeats this process until it provides a final answer, leveraging external tools like PDDL file generation and solving planners to achieve complex tasks dynamically.







